In [1]:
import time

from IPython.display import Markdown, display  # Import
from openai import OpenAI
import warnings
import tiktoken
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec
import os
from dotenv import load_dotenv
import asyncio
import pandas as pd
from io import StringIO
from src.business_requirements import *
from src.prompts import *
from src.simplified_business_requirements import *
from src.detailed_business_requirements import *
from src.permission_business_requirements import *
warnings.filterwarnings("ignore")
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
pinecone_api_key = os.getenv("PINECONE_API_KEY")


## Query Question

In [2]:
query = "What are the business requirements for Regulation O?"
response = verbatim_business_requirements(query, top_k=25)
display(Markdown(response))

"Business Requirement"^^"No member bank may extend credit to any insider of the bank or insider of its affiliates unless the extension of credit: (i) Is made on substantially the same terms (including interest rates and collateral) as, and following credit underwriting procedures that are not less stringent than, those prevailing at the time for comparable transactions by the bank with persons who are not covered by this part and who are not employed by the bank; and (ii) Does not involve more than the normal risk of repayment or present other unfavorable features."  
"No member bank may extend credit (which term includes granting a line of credit) to any insider of the bank or insider of its affiliates in an amount that, when aggregated with the amount of all other extensions of credit to that person and to all related interests of that person, exceeds the higher of $25,000 or 5 percent of the member bank's unimpaired capital and unimpaired surplus, unless: (i) the extension of credit has been approved in advance by a majority of the entire board of directors of that bank; and (ii) the interested party has abstained from participating directly or indirectly in the voting."  
"Upon receipt of a written request from the public, a member bank shall make available the names of each of its executive officers and each of its principal shareholders to whom, or to whose related interests, the member bank had outstanding as of the end of the latest previous quarter an extension of credit that, when aggregated with all other outstanding extensions of credit at such time from the member bank to such person and to all related interests of such person, equaled or exceeded 5 percent of the member bank's capital and unimpaired surplus or $500,000, whichever amount is less."  
"Each executive officer or director of a member bank the shares of which are not publicly traded shall report annually to the board of directors of the member bank the outstanding amount of any credit that was extended to the executive officer or director and that is secured by shares of the member bank."  
"Any member bank, or any officer, director, employee, agent, or other person participating in the conduct of the affairs of the bank, that violates any provision of this part (other than Sec. 215.9) is subject to civil penalties as specified in section 29 of the Federal Reserve Act (12 U.S.C. 504)."  
"The requirements of this part apply to savings associations, as defined in 12 CFR 238.2(l) (including any subsidiary of a savings association), in the same manner and to the same extent as if the savings association were a member bank; provided that a savings association's unimpaired capital and unimpaired surplus will be determined under regulatory capital rules applicable to that savings association."

## Create Excel Sheet

In [3]:
def clean_dataframe(df, column):
    """
    Cleans the DataFrame by:
      - Dropping rows in which the specified column does not contain any alphabetical characters (A-Z, a-z).
      - Removing the substring "| " from the specified column.

    Parameters:
      df (pd.DataFrame): The input DataFrame.
      column (str): The name of the column to clean.

    Returns:
      pd.DataFrame: The cleaned DataFrame.
    """
    # Ensure the column values are treated as strings.
    df[column] = df[column].astype(str)

    # Keep only rows where the column contains at least one alphabetical character.
    df = df[df[column].str.contains(r'[A-Za-z]', na=False)]

    # Remove the exact substring "| " from the column.
    df[column] = df[column].str.replace("| ", "", regex=False)

    # Remove the exact substring "|" from the column.
    df[column] = df[column].str.replace("|", "", regex=False)

    # Remove the exact substring "^^" from the column.
    df[column] = df[column].str.replace("^^", "", regex=False)

    # Remove the exact substring "" " from the column.
    df[column] = df[column].str.replace('''"''', "", regex=False)
    return df

df = convert_str_to_df(response)

# Clean Table
df = df.dropna(axis=1, how='all')
df = clean_dataframe(df, df.columns[0])
df["Business_Requirements"] = df[df.columns[0]]
df = df.drop(df.columns[0], axis=1)
df


,Business_Requirements
0,No member bank may extend credit (which term i...
1,Upon receipt of a written request from the pub...
2,Each executive officer or director of a member...
3,"Any member bank, or any officer, director, emp..."
4,The requirements of this part apply to savings...


## Create Simplified Business Requirements

In [4]:
async def simplified_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        simplified_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Simplified_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
df = await simplified_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements
0,No member bank may extend credit (which term i...,The business requirements for Regulation O sta...
1,Upon receipt of a written request from the pub...,The business requirement for Regulation O stat...
2,Each executive officer or director of a member...,The business requirement for Regulation O stat...
3,"Any member bank, or any officer, director, emp...",The business requirements for Regulation O sta...
4,The requirements of this part apply to savings...,The business requirements for Regulation O app...


## Obtain Detail Business Requirements

In [5]:
async def detailed_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        detailed_buss_req(original_question=query, business_requirement=row)
        for row in df["Business_Requirements"]
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the simplified business requirements back into the DataFrame
    df["Detailed_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
df = await detailed_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements
0,No member bank may extend credit (which term i...,The business requirements for Regulation O sta...,- **Identify Insiders**: Clearly define who qu...
1,Upon receipt of a written request from the pub...,The business requirement for Regulation O stat...,- **Receive Written Request**: Establish a pro...
2,Each executive officer or director of a member...,The business requirement for Regulation O stat...,- **Identify Executive Officers and Directors*...
3,"Any member bank, or any officer, director, emp...",The business requirements for Regulation O sta...,To address the Verbatim Business Requirements ...
4,The requirements of this part apply to savings...,The business requirements for Regulation O app...,To address the Verbatim Business Requirements ...


##

## Obtain Detailed Business Permissions

In [6]:
async def permissions_business_requirement(query: str, df: pd.DataFrame):
    # Create asynchronous tasks for each row's business requirement
    tasks = [
        permissions_buss_req(
            original_question=query,
            business_requirement=row["Business_Requirements"],
            detailed_business_requirement=row["Detailed_Business_Requirements"]
        )
        for _, row in df.iterrows()
    ]

    # Execute all tasks concurrently and collect results
    results = await asyncio.gather(*tasks)

    # Store the generated permissions back into the DataFrame
    df["Permission_Business_Requirements"] = results
    return df

# Run the asynchronous function with your query and DataFrame
df = await permissions_business_requirement(query=query, df=df)
df

,Business_Requirements,Simplified_Business_Requirements,Detailed_Business_Requirements,Permission_Business_Requirements
0,No member bank may extend credit (which term i...,The business requirements for Regulation O sta...,- **Identify Insiders**: Clearly define who qu...,- Banks are permitted to extend credit to insi...
1,Upon receipt of a written request from the pub...,The business requirement for Regulation O stat...,- **Receive Written Request**: Establish a pro...,### Permissions Based on Business Requirements...
2,Each executive officer or director of a member...,The business requirement for Regulation O stat...,- **Identify Executive Officers and Directors*...,### **Permissions Based on Provided Context an...
3,"Any member bank, or any officer, director, emp...",The business requirements for Regulation O sta...,To address the Verbatim Business Requirements ...,### Permissions Based on Regulation O\n\n- **E...
4,The requirements of this part apply to savings...,The business requirements for Regulation O app...,To address the Verbatim Business Requirements ...,- Savings associations and their subsidiaries ...


## Obtain Detailed Prohibitions

In [ ]:
e

In [7]:
df.to_excel("../Data/Results/Query_Results.xlsx")